In [1]:
import tensorflow as tf

# Variable sharing

In [3]:
def two_hidden_layers(x):
    w1 = tf.Variable(tf.random_normal([100, 50]), name='h1_weights')
    b1 = tf.Variable(tf.zeros([50]), name='h1_biases')
    h1 = tf.matmul(x, w1) + b1

    w2 = tf.Variable(tf.random_normal([50, 10]), name='h2_weights')
    b2 = tf.Variable(tf.zeros([10]), name='2_biases')
    logits = tf.matmul(h1, w2) + b2
    return logits

tf.reset_default_graph()

x1 = tf.Variable(tf.random_normal([200,100]))
x2 = tf.Variable(tf.random_normal([200,100]))
logits1 = two_hidden_layers(x1)
logits2 = two_hidden_layers(x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/var_sharing', sess.graph)

    sess.run([logits1, logits2])
                                    
writer.close()

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def two_hidden_layers(x):
    w1 = tf.get_variable("h1_weights", [100, 50], initializer=tf.random_normal_initializer())
    b1 = tf.get_variable('h1_biases', [50], initializer=tf.constant_initializer(0.0))
    h1 = tf.matmul(x, w1) + b1

    w2 = tf.get_variable("h2_weights", [50, 10], initializer=tf.random_normal_initializer())
    b2 = tf.get_variable('h2_biases', [10], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(h1, w2) + b2
    return logits

tf.reset_default_graph()

x1 = tf.Variable(tf.random_normal([200,100]))
x2 = tf.Variable(tf.random_normal([200,100]))
# logits1 = two_hidden_layers(x1)
# logits2 = two_hidden_layers(x2) # ValueError: Variable h1_weights already exists, disallowed. Did you mean to set reuse=True in VarScope?

with tf.variable_scope("two_layers") as scope:
    logits1 = two_hidden_layers(x1)
    scope.reuse_variables()
    logits2 = two_hidden_layers(x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/var_sharing', sess.graph)

    sess.run([logits1, logits2])
                                    
writer.close()

In [14]:
def fully_connected(x, output_dim, scope):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE) as scope:
        w = tf.get_variable("weights", [x.shape[1], output_dim], initializer=tf.random_normal_initializer())
        b = tf.get_variable("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        return tf.matmul(x, w) + b

def two_hidden_layers(x):
    h1 = fully_connected(x, 50, "h1")
    return fully_connected(h1, 10, "h2")
    

tf.reset_default_graph()
x1 = tf.Variable(tf.random_normal([200,100]))
x2 = tf.Variable(tf.random_normal([200,100]))
with tf.variable_scope("two_layers") as scope:
    logits1 = two_hidden_layers(x1)
    logits2 = two_hidden_layers(x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/var_sharing', sess.graph)
    sess.run([logits1, logits2])
                                    
writer.close()

# Save and Restore

In [ ]:
# tf.summary.scalar("loss", self.loss)
# tf.summary.histogram("histogram loss", self.loss)
# summary_op = tf.summary.merge_all()

# saver = tf.train.Saver() # defaults to saving all variables

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
#     if ckpt and ckpt.model_checkpoint_path:
#         saver.restore(sess, ckpt.model_checkpoint_path)

#     writer = tf.summary.FileWriter('./graphs', sess.graph)
#     for index in range(10000):
#         ...
#         loss_batch, _, summary = sess.run([loss, optimizer, summary_op])
#         writer.add_summary(summary, global_step=index)

#         if (index + 1) % 1000 == 0:
#             saver.save(sess, 'checkpoints/skip-gram', index)

# Control Randomization

In [4]:
c = tf.random_uniform([], -10, 10, seed=2)
with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    print(sess.run(c)) # >> -5.97319

3.574932
-5.9731865


In [5]:
c = tf.random_uniform([], -10, 10, seed=2)
with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493    
with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493

3.574932
3.574932


In [7]:
c = tf.random_uniform([], -10, 10, seed=2)
d = tf.random_uniform([], -10, 10, seed=2)
with tf.Session() as sess:
    print(sess.run(c)) # >> 3.57493
    print(sess.run(d)) # >> 3.57493

3.574932
3.574932


In [15]:
tf.set_random_seed(2)
c = tf.random_uniform([], -10, 10)
d = tf.random_uniform([], -10, 10)

with tf.Session() as sess:
    print(sess.run(c)) # >> -4.00752
    print(sess.run(d)) # >> -2.98339

6.8672733
0.98825264
